In [ ]:
from pathlib import Path
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import h5py
import copy
import pickle

# Project modules
from lib.Stokes import Stokes
from functions.plot_data import plot_data
from functions.plot_angle_gradient import plot_angle_gradient

In [ ]:
# Open data files
from functions.load_pickles import load_pickles
stokes_list, _ = load_pickles(select="stokes")

In [ ]:
# Extract each Stokes parameter into dictionary, to make it easier to work with
I = stokes_list['I']
Q = stokes_list['Q']
U = stokes_list['U']
V = stokes_list['V']

In [ ]:
# Define function to sum data along the third axis (sum all wavelengths)
def sw(data):
    return np.sum(data, axis=2)

In [ ]:
# Calculate the vertical (longitudinal) component

#NOTE: these values are divided into _1 for the first spectral line and _2 for the second spectral line
lambda0  = [6301.51, 6302.50]  # in Angstroms
gbar     = [1.669, 2.487] # Lozitsky
C1 = [4.6686e-13 * l**2 * g for l, g in zip(lambda0, gbar)]
print(f"C1_1: {C1[0]}, C1_2: {C1[1]}")

f = 1 # filling factor, assumed to be 1

line_cuttoff = 55 # index at which we divide the data, between the spectral lines



# Calculate the horizontal (transverse) component

Gbar = [g**2 for g in gbar]  # Landi Degl'Innocenti & Landolfi (2004), si la línea es un triplete #TODO
C2 = [5.4490e-26 * l**4 * g for l, g in zip(lambda0, Gbar)]
print(f"C2_1: {C2[0]}, C2_2: {C2[1]}")

print(Gbar)

# Strong field approximation
C = 4.67e-13 # TODO: where is this constant from? Sara's email (and PDF) 29/08


In [ ]:
Bv = [-sw((V.data_n * I.data_d)[:, :, :line_cuttoff]) / (C1[0] * f * sw((I.data_d ** 2)[:, :, :line_cuttoff])),
       -sw((V.data_n * I.data_d)[:, :, line_cuttoff:]) / (C1[1] * f * sw((I.data_d ** 2)[:, :, line_cuttoff:]))]

print(f'Mean Bv [G]:\nFirst line: {np.mean(Bv[0])}\nSecond line: {np.mean(Bv[1])}')

In [ ]:
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin, vmax, midpoint=0, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        normalized_min = max(0, 1 / 2 * (1 - abs((self.midpoint - self.vmin) / (self.midpoint - self.vmax))))
        normalized_max = min(1, 1 / 2 * (1 + abs((self.vmax - self.midpoint) / (self.midpoint - self.vmin))))
        normalized_mid = 0.5
        x, y = [self.vmin, self.midpoint, self.vmax], [normalized_min, normalized_mid, normalized_max]
        return np.ma.masked_array(np.interp(value, x, y))


In [ ]:
divnorm=MidpointNormalize(vmin=-3000, vmax=4000, midpoint=0)
plot_data(Bv[0], colourmap='berlin_r', norm=divnorm, colourbar_label=r'$B_{||}$ [G]')

In [ ]:
L = np.sqrt(Q.data_n**2 + U.data_n**2)
Bt = [sw(L[:,:,:line_cuttoff] * np.abs(I.data_dd[:,:,:line_cuttoff])) / (C2[0] * f * sw(np.abs(I.data_dd[:,:,:line_cuttoff]))**2),
       sw(L[:,:,line_cuttoff:] * np.abs(I.data_dd[:,:,line_cuttoff:])) / (C2[1] * f * sw(np.abs(I.data_dd[:,:,line_cuttoff:]))**2)]
Bt = np.sqrt(Bt)


print(f'Mean Bt [G]:\nFirst line: {np.mean(Bt[0])}\nSecond line: {np.mean(Bt[1])}')

In [ ]:
divnorm=MidpointNormalize(vmin=50, vmax=500, midpoint=0)

plot_data(Bt[0], colourmap='berlin_r', norm=divnorm, colourbar_label=r'$B_{\perp}$ [G]')

In [ ]:
B_WFA = np.sqrt(Bv[0]**2 + Bt[0]**2)
print(f'Mean B [G]: {np.mean(B_WFA)}')

divnorm=MidpointNormalize(vmin=-3000, vmax=4000, midpoint=0)
plot_data(B_WFA, colourmap='berlin_r', norm=divnorm, colourbar_label=r'$B$ [G]')

In [ ]:
# Calculate inclination angle

const = [4/3 * g**2/G for g, G in (gbar, Gbar)]

num = [sw(np.abs(I.wave_array[:line_cuttoff] - lambda0[0]) * np.abs(L[:,:,:line_cuttoff]) * V.data_n[:,:,:line_cuttoff]**2 * np.abs(I.data_d[:,:,:line_cuttoff])),
         sw(np.abs(I.wave_array[line_cuttoff:] - lambda0[1]) * np.abs(L[:,:,line_cuttoff:]) * V.data_n[:,:,line_cuttoff:]**2 * np.abs(I.data_d[:,:,line_cuttoff:]))]
denom = [sw(np.abs(V.data_n[:,:,:line_cuttoff])**4),
         sw(np.abs(V.data_n[:,:,line_cuttoff:])**4)]
tan2gamma = [const[0] * num[0] / denom[0],
             const[1] * num[1] / denom[1]]
gamma_mod = np.arctan(np.sqrt(tan2gamma))

In [ ]:
shape = I.data_n[:, :, 0].shape
sign_gamma = np.zeros(shape)
gamma = np.copy(gamma_mod[0])

for i in range(0, shape[0]):
    for j in range(0, shape[1]):
        x = I.wave_array[:line_cuttoff]
        y = V.data_n[i,j,:line_cuttoff]

        # Find wavelength of peak
        peak = np.argmax(y)

        # Sign comes from whether peak is blue or redshifted with respect to line center
        sign_gamma[i,j] = x[peak] < lambda0[0]

        if sign_gamma[i,j] == 0:
            gamma[i,j] = np.pi - gamma[i,j]

In [ ]:
plot_angle_gradient(gamma, colourbar_label=r'$\gamma$ [deg]')

In [ ]:
# Compute the inclination angle, simpler
theta = np.arctan(Bt[0] / Bv[0])

In [ ]:
# Set all negative values of theta to theta + pi
theta[theta < 0] += np.pi

In [ ]:
plot_angle_gradient(theta, colourbar_label=r'$\theta$ [deg]')
plot_angle_gradient(gamma, colourbar_label=r'$\gamma$ [deg]')

In [ ]:
# Compute the azimuth angle (equal for both lines, does not depend on wavelength or Landé factor)
chi_mod = [np.arctan(sw(U.data_n[:, :, :line_cuttoff] * I.data_dd[:, :, :line_cuttoff]) / sw(Q.data_n[:, :, :line_cuttoff] * I.data_dd[:, :, :line_cuttoff])) / 2,
    np.arctan(sw(U.data_n[:, :, line_cuttoff:] * I.data_dd[:, :, line_cuttoff:]) / sw(Q.data_n[:, :, line_cuttoff:] * I.data_dd[:, :, line_cuttoff:])) / 2]

In [ ]:
# Find sign of blue lobe peak
shape = np.shape(I.data_n[:,:,0])
sign_chi = np.zeros(shape)
chi = np.copy(chi_mod[0])

for i in range(0, shape[0]):
    for j in range(0, shape[1]):
        # Sign comes from sign of the vertical component
        sign_chi[i,j] = np.sign(Bv[0][i,j])

        if sign_chi[i,j] < 0:
            chi[i,j] = np.pi - chi[i,j]

In [ ]:
plot_angle_gradient(chi, colourbar_label=r'$\chi$ [deg]')

In [ ]:
num = sw(U.data_n[:, :, :line_cuttoff] * I.data_dd[:, :, :line_cuttoff])
den = sw(Q.data_n[:, :, :line_cuttoff] * I.data_dd[:, :, :line_cuttoff])
phi = 0.5 * np.arctan(num/den)

In [ ]:
for i in range(0, shape[0]):
    for j in range(0, shape[1]):
        if den[i,j] < 0:
            phi[i,j] += np.pi/2
        elif num[i,j] < 0 and den[i,j] > 0:
            phi[i,j] += 2*np.pi
        if den[i,j] == 0:
            if num[i,j] > 0:
                phi[i,j] += np.pi/4
            elif num[i,j] < 0:
                phi[i,j] += 3/4*np.pi

In [ ]:
plot_angle_gradient(chi, colourbar_label=r'$\phi$ [deg]', scale=[0,180])

In [ ]:
# Downsample phi by averaging 20x20 pixel blocks
def block_average(arr, block_size):
    ny, nx = arr.shape
    by = ny // block_size
    bx = nx // block_size
    arr_cropped = arr[:by*block_size, :bx*block_size]
    arr_reshaped = arr_cropped.reshape(by, block_size, bx, block_size)
    arr_blocked = arr_reshaped.mean(axis=(1,3))
    return arr_blocked

phi_blocked = block_average(phi, 20)

In [ ]:
plot_angle_gradient(phi_blocked, colourbar_label=r'$\phi$ [deg]', scale=[0,180])

In [ ]:
# Plot phi_blocked as lines with inclination representing the angle
fig, ax = plt.subplots(figsize=(8,8))

# extent=[0, 89.88485, 0, 122.88] %TODO
divnorm=MidpointNormalize(vmin=-3000, vmax=4000, midpoint=0)
img = ax.imshow(B_WFA, cmap='grey', origin='lower')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax, label='B [Gauss]')

nrows, ncols = phi_blocked.shape
block_size = 20
for i in range(nrows):
    for j in range(ncols):
        # Center of each block
        y = i * block_size + block_size/2
        x = j * block_size + block_size/2
        angle = phi_blocked[i, j]
        # Length of the line
        length = block_size * 0.8
        # Convert angle from degrees to radians if needed
        theta_rad = angle
        # The angle phi is usually measured from the +x axis, but matplotlib's y axis increases downward.
        # To match image orientation, flip the sign of sin component:
        dx = length/2 * np.cos(theta_rad)
        dy = length/2 * np.sin(theta_rad)
        ax.plot([x-dx, x+dx], [y-dy, y+dy], color='red', lw=0.8)
ax.set_aspect('equal')
ax.set_xlim(0, ncols*block_size)
ax.set_ylim(0, nrows*block_size)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title(r'Inclination map from $\phi$ (block-averaged)')
plt.show()

In [ ]:
def noise_level(a):
    a = np.asanyarray(a)
    sd = a.std(ddof=1)
    return sd

# Example spectrum noise level
spectrum = copy.copy(V.data_n[300,300,:])
sd = noise_level(spectrum[90:])
print(f'Noise level = {sd}')

In [ ]:
# Calculate delta lambda_B for each pixel, for each line

dlB = np.zeros(I.data.shape)[:,:,:2].astype(float)
dlB_binary = np.zeros(I.data.shape)[:,:,:2].astype(float)
margin = 3.5
verbose = 0

for i in range(0, np.shape(V.data_n)[0]):

    # Read out every 50 rows
    if np.mod(i,50) == 0:
        print(f'Row {i} of {np.shape(V.data_n)[0]}')

    for j in range(0, np.shape(V.data_n)[1]):
        if (verbose):
            print(f'For pixel {i},{j}:')

        # Initialize variables
        peaks_p = [0, 0]
        peaks_n = [0, 0]

        # Get the spectrum for this pixel
        spectrum = copy.copy(V.data_n[i,j,:])

        # Calculate noise level for spectrum region outside the spectral lines
        sd = noise_level(spectrum[90:])
        if (verbose):
            print(f'Noise level = {sd}')

        # First line
        peaks_p[0] = int(np.argmax(spectrum[:60])) if spectrum[:60].max() > margin*sd else None
        peaks_n[0] = int(np.argmin(spectrum[:60])) if spectrum[:60].min() < -margin*sd else None

        # # Second line
        peaks_p[1] = 60 + int(np.argmax(spectrum[60:])) if spectrum[60:].max() > margin*sd else None
        peaks_n[1] = 60 + int(np.argmin(spectrum[60:])) if spectrum[60:].min() < -margin*sd else None

        if (verbose):
            print(f'Peak positions: {peaks_p}, {peaks_n}')

        # If two peaks have been found for the first line, calculate dlB
        if peaks_p[0] is not None and peaks_n[0] is not None:
            dlB_binary[i, j, 0] = 1
            dlB[i, j, 0] = np.array(V.wave_array)[peaks_p[0]] - np.array(V.wave_array)[peaks_n[0]]

            if (verbose):
                plt.figure()
                plt.vlines(V.wave_array[peaks_n[0]], ymin=np.min(spectrum), ymax=np.max(spectrum), color='g', linestyle='--')
                plt.vlines(V.wave_array[peaks_p[0]], ymin=np.min(spectrum), ymax=np.max(spectrum), color='purple', linestyle='--')

                print(f'Peak 1 positive: {V.wave_array[peaks_p[0]]}')
                print(f'Peak 1 negative: {V.wave_array[peaks_n[0]]}')
                print(np.array(V.wave_array)[peaks_p[0]] - np.array(V.wave_array)[peaks_n[0]])
                print(f'Distance: {dlB[i, j, 0]:.6f} Angstrom')
        else:
            dlB[i, j, 0] = np.nan


        # If two peaks have been found for the second line, calculate dlB
        if peaks_p[1] is not None and peaks_n[1] is not None:
            dlB_binary[i, j, 1] = 1
            dlB[i, j, 1] = np.array(V.wave_array)[peaks_p[1]] - np.array(V.wave_array)[peaks_n[1]]

            if (verbose):
                plt.figure()
                plt.vlines(V.wave_array[peaks_n[1]], ymin=np.min(spectrum), ymax=np.max(spectrum), color='g', linestyle='--')
                plt.vlines(V.wave_array[peaks_p[1]], ymin=np.min(spectrum), ymax=np.max(spectrum), color='purple', linestyle='--')

                print(f'Peak 2 positive: {V.wave_array[peaks_p[1]]}')
                print(f'Peak 2 negative: {V.wave_array[peaks_n[1]]}')
                print(np.array(V.wave_array)[peaks_p[1]] - np.array(V.wave_array)[peaks_n[1]])
                print(f'Distance: {dlB[i, j, 1]:.6f} Angstrom')
        else:
            dlB[i, j, 1] = np.nan

        if (verbose):
            plt.hlines(margin*sd, V.wave_array[0], V.wave_array[-1], color='red', linestyle='--')
            plt.hlines(-margin*sd, V.wave_array[0], V.wave_array[-1], color='red', linestyle='--')
            plt.plot(V.wave_array, spectrum, linestyle='-', color='black')

In [ ]:
# Select pixels of interest
verbose=1
plate_scale_x = 0.14857 # arcseconds per pixel
plate_scale_y = 0.16 # arcseconds per pixel
x_pix = np.array([500, 180, 290, 400, 320, 300])
y_pix = np.array([118, 113, 538, 354, 428, 413])
pix_name=['A', 'B', 'C', 'D', 'E', 'F']
margin = 3

fig, axs = plt.subplots(3, 2, figsize=(9, 7.5))

for i in range(6):

    # Initialize variables
    peaks_p = [0, 0]
    peaks_n = [0, 0]

    # Get the spectrum for this pixel
    spectrum = copy.copy(V.data_n[y_pix[i],x_pix[i],:])

    # Calculate noise level for spectrum region outside the spectral lines
    sd = noise_level(spectrum[90:])

    # First line
    peaks_p[0] = int(np.argmax(spectrum[:60])) if spectrum[:60].max() > margin*sd else None
    peaks_n[0] = int(np.argmin(spectrum[:60])) if spectrum[:60].min() < -margin*sd else None

    # # Second line
    peaks_p[1] = 60 + int(np.argmax(spectrum[60:])) if spectrum[60:].max() > margin*sd else None
    peaks_n[1] = 60 + int(np.argmin(spectrum[60:])) if spectrum[60:].min() < -margin*sd else None

    if (verbose):
        row, col = divmod(i, 2)
        ax = axs[row, col]
        ax.set_title(f'Pixel {pix_name[i]}: '+r'$\Delta \lambda_B$' f'= [{dlB[y_pix[i], x_pix[i], 0]:.4f}, {dlB[y_pix[i], x_pix[i], 1]:.4f}] ' + r'$\AA$')

        # Only plot if there are no None values in peaks_p or peaks_n
        if all(p is not None for p in peaks_p) and all(n is not None for n in peaks_n):
            for k in range(2):
                ax.axvline(V.wave_array[peaks_p[k]], color='purple', linewidth=0.8)
                ax.axvline(V.wave_array[peaks_n[k]], color='green', linewidth=0.8)

        ax.plot(I.wave_array, V.data_n[y_pix[i],x_pix[i],:], label='I data', linewidth=0.4, marker='.', markersize=5)
        # Remove inner axes
        if row < 2 and col < 2:
            ax.set_xticks([])
        else:
            ax.set_xlabel(r'Wavelength [$\AA$]')

        ax.set_ylabel('V')

axs[0, 0].set_title('Pixel A (no peaks detected)')
plt.tight_layout()
plt.show()

In [ ]:
# Calculate strong magnetic field
B_strong = dlB / (2 * C * np.array(lambda0)**2 * np.array(gbar))

In [ ]:
Bv = np.moveaxis(Bv, 0, -1)
Bt = np.moveaxis(Bt, 0, -1)

In [ ]:
# Constants
kB = 1.3806488e-16 # [erg K-1]
h = 6.6260755e-27  # [erg s]
c = 2.99792458e10  # [cm · s−1]
lambda0  = np.array([6301.51*1e-8, 6302.50*1e-8])   # Angstroms to cm
l = np.mean(lambda0)
Icont = I.data_n[:,:,:5].mean(axis=2) # I map in continuum
Teff = 5780 # [K] T quiet sun average
T = (1/Teff - kB*l/(h*c) * np.log(Icont))**-1
M = 55.845 # Fe atomic mass, [g mol-1]
av = 6.022e23 # avogadro, [mol-1]
m =  M/av
Xi = 0 # microturbulence, assumed 0

# Rebecca Centeno, equation 3
dlD = np.array([6301.5/c * np.sqrt(2*kB*T/m + Xi**2), 6302.5/c * np.sqrt(2*kB*T/m + Xi**2)]) # will be in units of lambda, in this case Angstrom

dlD = np.moveaxis(dlD, 0, -1)
np.shape(dlD)


In [ ]:
# remove outliers
B_strong[B_strong > 5000] = np.nan
B_strong[B_strong < -5000] = np.nan

In [ ]:
B = np.empty(Bv.shape)
B[:,:,:] = np.nan
B_mask = np.zeros(Bv.shape) * np.nan
# gamma = np.zeros(derived.weak.Bv.shape) * np.nan
# chi = np.zeros(derived.weak.Bv.shape) * np.nan

for i in range(I.data.shape[0]):
    for j in range(I.data.shape[1]):
        for k in range(0,2):
            if (dlB_binary[i,j,k] == 0) or np.isnan(dlB[i,j,k]) or (np.abs(dlB[i,j,k]) < np.abs(dlD[i,j,k])):
                # If dlB could not be calculated, or if it is less than the doppler effect
                B[i,j,k] = np.sqrt(Bv[i,j,k]**2 + Bt[i,j,k]**2)
                B_mask[i,j,k] = 0
                # chi[i,j,k] = derived.weak.chi[i,j,k]
                # gamma[i,j,k] = derived.weak.gamma[i,j,k]
            else:
                # Use strong field approximation otherwise
                B[i,j,k] = B_strong[i,j,k]
                B_mask[i,j,k] = 1
                # chi[i,j,k] = derived.strong.chi[i,j,k]
                # gamma[i,j,k] = derived.strong.gamma[i,j,k]

In [ ]:
print(np.abs(dlD[300,300,0]))
print(np.abs(dlB[300,300,0]))

In [ ]:
# Histogram of derived.strong.B[:,:,0]
plt.figure()
plt.hist(dlB[:,:,0].ravel(), bins=100, color='blue', alpha=0.7)
plt.xlabel('Magnetic field strength (first line)')
plt.ylabel('Count')
plt.title('Histogram of dlB[:,:,0]')
plt.show()

In [ ]:
# Histogram of derived.strong.B[:,:,0]
plt.figure()
plt.hist(B[:,:,0].ravel(), bins=100, color='blue', alpha=0.7)
plt.xlabel('Magnetic field strength (first line)')
plt.ylabel('Count')
plt.title('Histogram of B[:,:,0]')
plt.show()

In [ ]:
plot_data(dlB[:,:,0], colourmap='berlin', title=r"$\Delta \lambda _B$, 6301.5 $\mathrm{\AA}$, in $\mathrm{\AA}$")

In [ ]:
plot_data(B[:,:,0], colourmap='bwr', title=r"B combined [G]")

In [ ]:
plot_data(dlB[:,:,0] > dlD[:,:,0], colourmap='grey')